In [39]:
import nltk
import os
import re
import bs4
import requests
from gensim.summarization import keywords
import math
import operator
import numpy as np
import pandas as pd

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords

Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()

In [5]:
subject = input("enter the wikipedia topic to be summarised ")
base_url = "https://en.wikipedia.org/wiki/" + subject
page = requests.get(base_url)

soup = bs4.BeautifulSoup(page.content, 'html.parser')
paragraphs = soup.find_all('p')

content = ' '
for paragraph in paragraphs: 
    content += paragraph.text
content

enter the wikipedia topic to be summarised Natural Language Processing


' Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.  The result is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them. The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves. \nChallenges in natural language processing frequently involve speech recognition, natural language understanding, and natural-language generation.\nNatural language processing has its roots in the 1950s. Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, a task that involves the automated 

In [6]:
#tokenization 
from nltk.tokenize import word_tokenize,sent_tokenize
sentences = sent_tokenize(content)
sentences

[' Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.',
 'The result is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them.',
 'The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves.',
 'Challenges in natural language processing frequently involve speech recognition, natural language understanding, and natural-language generation.',
 'Natural language processing has its roots in the 1950s.',
 'Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, a task that involv

In [7]:
# removing punctuation, numbers and special characters, also converting to lower case
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
clean_sentences = [s.lower() for s in clean_sentences]
clean_sentences[1]

<ipython-input-7-129be891b5ae>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


'the result is a computer capable of  understanding  the contents of documents  including the contextual nuances of the language within them '

In [8]:
#importing stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [9]:
#removing stop words
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
clean_sentences[:10]

['natural language processing nlp subfield linguistics computer science artificial intelligence concerned interactions computers human language particular program computers process analyze large amounts natural language data',
 'result computer capable understanding contents documents including contextual nuances language within',
 'technology accurately extract information insights contained documents well categorize organize documents',
 'challenges natural language processing frequently involve speech recognition natural language understanding natural language generation',
 'natural language processing roots',
 'already alan turing published article titled computing machinery intelligence proposed called turing test criterion intelligence task involves automated interpretation generation natural language time articulated problem separate artificial intelligence',
 'premise symbolic nlp well summarized john searle chinese room experiment given collection rules e g chinese phrasebook 

In [10]:
tokenized_words = []
for sentence in clean_sentences: 
    tokenized_words.append(nltk.word_tokenize(sentence))

In [11]:
tokenized_word_results = []
for arr in tokenized_words:
    tokenized_word_results.extend(arr)

In [12]:
tokenized_words

[['natural',
  'language',
  'processing',
  'nlp',
  'subfield',
  'linguistics',
  'computer',
  'science',
  'artificial',
  'intelligence',
  'concerned',
  'interactions',
  'computers',
  'human',
  'language',
  'particular',
  'program',
  'computers',
  'process',
  'analyze',
  'large',
  'amounts',
  'natural',
  'language',
  'data'],
 ['result',
  'computer',
  'capable',
  'understanding',
  'contents',
  'documents',
  'including',
  'contextual',
  'nuances',
  'language',
  'within'],
 ['technology',
  'accurately',
  'extract',
  'information',
  'insights',
  'contained',
  'documents',
  'well',
  'categorize',
  'organize',
  'documents'],
 ['challenges',
  'natural',
  'language',
  'processing',
  'frequently',
  'involve',
  'speech',
  'recognition',
  'natural',
  'language',
  'understanding',
  'natural',
  'language',
  'generation'],
 ['natural', 'language', 'processing', 'roots'],
 ['already',
  'alan',
  'turing',
  'published',
  'article',
  'titled',


In [13]:
def lemmatize_words(words):
    lemmatized_words = []
    for word in words:
        lemmatized_words.append(wordlemmatizer.lemmatize(word))
    return lemmatized_words

In [14]:
lemmatized_words = lemmatize_words(tokenized_word_results)

In [15]:
lemmatize_words

<function __main__.lemmatize_words(words)>

In [16]:
len(lemmatized_words)
len(tokenized_word_results)

823

In [17]:
def freq(words):
    dict_freq = {}
    words_unique = []
    for word in words:
        if word not in words_unique:
            words_unique.append(word)
    for word in words_unique:
        dict_freq[word] = words.count(word)
    return dict_freq

In [18]:
word_freq = freq(tokenized_word_results)

In [19]:
word_freq

{'natural': 20,
 'language': 28,
 'processing': 16,
 'nlp': 17,
 'subfield': 1,
 'linguistics': 9,
 'computer': 4,
 'science': 3,
 'artificial': 2,
 'intelligence': 4,
 'concerned': 1,
 'interactions': 1,
 'computers': 2,
 'human': 2,
 'particular': 1,
 'program': 1,
 'process': 2,
 'analyze': 2,
 'large': 3,
 'amounts': 1,
 'data': 5,
 'result': 1,
 'capable': 1,
 'understanding': 4,
 'contents': 1,
 'documents': 4,
 'including': 1,
 'contextual': 1,
 'nuances': 1,
 'within': 1,
 'technology': 1,
 'accurately': 1,
 'extract': 1,
 'information': 1,
 'insights': 1,
 'contained': 1,
 'well': 2,
 'categorize': 1,
 'organize': 1,
 'challenges': 1,
 'frequently': 2,
 'involve': 2,
 'speech': 4,
 'recognition': 2,
 'generation': 2,
 'roots': 1,
 'already': 1,
 'alan': 1,
 'turing': 2,
 'published': 1,
 'article': 1,
 'titled': 1,
 'computing': 1,
 'machinery': 1,
 'proposed': 1,
 'called': 2,
 'test': 1,
 'criterion': 1,
 'task': 3,
 'involves': 1,
 'automated': 1,
 'interpretation': 1,
 'ti

In [20]:
def pos_tagging(text):
    pos_tag = nltk.pos_tag(text.split())
    pos_tagged_noun_verb = []
    for word,tag in pos_tag:
        if tag == "NN" or tag == "NNP" or tag == "NNS" or tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
            pos_tagged_noun_verb.append(word)
    print(pos_tagged_noun_verb)
    return pos_tagged_noun_verb

In [21]:
def word_tfidf(dict_freq,word,sentences,sentence):
    word_tfidf = [] 
    tf = tf_score(word,sentence)
    idf = idf_score(len(sentences),word,sentences)
    tf_idf = tf_idf_score(tf,idf)
    return tf_idf

In [22]:
def tf_score(word,sentence):
    print(f'Sentence from tf_Score {sentence}, word {word}')
    freq_sum = 0
    word_frequency_in_sentence = 0
    len_sentence = len(sentence)
    for word_in_sentence in sentence.split():
        if word == word_in_sentence:
            word_frequency_in_sentence = word_frequency_in_sentence + 1
    tf =  word_frequency_in_sentence/ len_sentence
    return tf

In [23]:
def idf_score(no_of_sentences,word,sentences):
    no_of_sentence_containing_word = 0
    for sentence in sentences:
        #sentence = remove_special_characters(str(sentence))
        #sentence = re.sub(r'\d+', '', sentence)
        sentence = sentence.split()
        sentence = [word for word in sentence if word.lower() not in Stopwords and len(word)>1]
        sentence = [word.lower() for word in sentence]
        sentence = [wordlemmatizer.lemmatize(word) for word in sentence]
        if word in sentence:
            no_of_sentence_containing_word = no_of_sentence_containing_word + 1
    idf = math.log10(no_of_sentences/no_of_sentence_containing_word)
    return idf

In [24]:
def tf_idf_score(tf,idf):
    return tf*idf

In [25]:
clean_sentences[:2]

['natural language processing nlp subfield linguistics computer science artificial intelligence concerned interactions computers human language particular program computers process analyze large amounts natural language data',
 'result computer capable understanding contents documents including contextual nuances language within']

In [26]:
def sentence_importance(sentence,dict_freq,sentences):
    sentence_score = 0
    pos_tagged_sentence = []
    no_of_sentences = len(sentences)
    pos_tagged_sentence = pos_tagging(sentence)
    for word in pos_tagged_sentence:
        if word.lower() not in Stopwords and word not in Stopwords    and len(word)>1:
            word = word.lower() 
            word = wordlemmatizer.lemmatize(word)
            sentence_score = sentence_score + word_tfidf(dict_freq,word,sentences,sentence)
    return sentence_score

In [30]:
c = 1
sentence_with_importance = {}
for sent in clean_sentences:
    sentenceimp = sentence_importance(sent,word_freq,clean_sentences)
    sentence_with_importance[c] = sentenceimp
    c = c+1
sentence_with_importance = sorted(sentence_with_importance.items(), key=operator.itemgetter(1),reverse=True)

['language', 'processing', 'subfield', 'linguistics', 'computer', 'science', 'intelligence', 'concerned', 'interactions', 'computers', 'language', 'program', 'computers', 'process', 'amounts', 'language', 'data']
Sentence from tf_Score natural language processing nlp subfield linguistics computer science artificial intelligence concerned interactions computers human language particular program computers process analyze large amounts natural language data, word language
Sentence from tf_Score natural language processing nlp subfield linguistics computer science artificial intelligence concerned interactions computers human language particular program computers process analyze large amounts natural language data, word processing
Sentence from tf_Score natural language processing nlp subfield linguistics computer science artificial intelligence concerned interactions computers human language particular program computers process analyze large amounts natural language data, word subfield
Se

Sentence from tf_Score challenges natural language processing frequently involve speech recognition natural language understanding natural language generation, word language
Sentence from tf_Score challenges natural language processing frequently involve speech recognition natural language understanding natural language generation, word generation
['language', 'processing', 'roots']
Sentence from tf_Score natural language processing roots, word language
Sentence from tf_Score natural language processing roots, word processing
Sentence from tf_Score natural language processing roots, word root
['turing', 'published', 'article', 'titled', 'computing', 'machinery', 'intelligence', 'proposed', 'called', 'turing', 'test', 'criterion', 'intelligence', 'task', 'involves', 'automated', 'interpretation', 'generation', 'language', 'time', 'articulated', 'problem', 'intelligence']
Sentence from tf_Score already alan turing published article titled computing machinery intelligence proposed called 

Sentence from tf_Score premise symbolic nlp well summarized john searle chinese room experiment given collection rules e g chinese phrasebook questions matching answers computer emulates natural language understanding nlp tasks applying rules data confronted, word computer
Sentence from tf_Score premise symbolic nlp well summarized john searle chinese room experiment given collection rules e g chinese phrasebook questions matching answers computer emulates natural language understanding nlp tasks applying rules data confronted, word emulates
Sentence from tf_Score premise symbolic nlp well summarized john searle chinese room experiment given collection rules e g chinese phrasebook questions matching answers computer emulates natural language understanding nlp tasks applying rules data confronted, word language
Sentence from tf_Score premise symbolic nlp well summarized john searle chinese room experiment given collection rules e g chinese phrasebook questions matching answers computer 

Sentence from tf_Score representation learning deep neural network style machine learning methods became widespread natural language processing due part flurry results showing techniques achieve state art results many natural language tasks example language modeling parsing many others, word method
Sentence from tf_Score representation learning deep neural network style machine learning methods became widespread natural language processing due part flurry results showing techniques achieve state art results many natural language tasks example language modeling parsing many others, word became
Sentence from tf_Score representation learning deep neural network style machine learning methods became widespread natural language processing due part flurry results showing techniques achieve state art results many natural language tasks example language modeling parsing many others, word language
Sentence from tf_Score representation learning deep neural network style machine learning methods 

Sentence from tf_Score early days many language processing systems designed symbolic methods e hand coding set rules coupled dictionary lookup writing grammars devising heuristic rules stemming, word method
Sentence from tf_Score early days many language processing systems designed symbolic methods e hand coding set rules coupled dictionary lookup writing grammars devising heuristic rules stemming, word hand
Sentence from tf_Score early days many language processing systems designed symbolic methods e hand coding set rules coupled dictionary lookup writing grammars devising heuristic rules stemming, word coding
Sentence from tf_Score early days many language processing systems designed symbolic methods e hand coding set rules coupled dictionary lookup writing grammars devising heuristic rules stemming, word set
Sentence from tf_Score early days many language processing systems designed symbolic methods e hand coding set rules coupled dictionary lookup writing grammars devising heuristi

...

Sentence from tf_Score however part speech tagging introduced use hidden markov models natural language processing increasingly research focused statistical models make soft probabilistic decisions based attaching real valued weights features making input data, word make
Sentence from tf_Score however part speech tagging introduced use hidden markov models natural language processing increasingly research focused statistical models make soft probabilistic decisions based attaching real valued weights features making input data, word decision
Sentence from tf_Score however part speech tagging introduced use hidden markov models natural language processing increasingly research focused statistical models make soft probabilistic decisions based attaching real valued weights features making input data, word based
Sentence from tf_Score however part speech tagging introduced use hidden markov models natural language processing increasingly research focused statistical models make soft proba

['areas', 'shift', 'entailed', 'changes', 'nlp', 'systems', 'designed', 'network', 'based', 'approaches', 'viewed', 'paradigm', 'language', 'processing']
Sentence from tf_Score areas shift entailed substantial changes nlp systems designed deep neural network based approaches may viewed new paradigm distinct statistical natural language processing, word area
Sentence from tf_Score areas shift entailed substantial changes nlp systems designed deep neural network based approaches may viewed new paradigm distinct statistical natural language processing, word shift
Sentence from tf_Score areas shift entailed substantial changes nlp systems designed deep neural network based approaches may viewed new paradigm distinct statistical natural language processing, word entailed
Sentence from tf_Score areas shift entailed substantial changes nlp systems designed deep neural network based approaches may viewed new paradigm distinct statistical natural language processing, word change
Sentence from t

Sentence from tf_Score latest works tend use non technical structure given task build proper neural network, word task
Sentence from tf_Score latest works tend use non technical structure given task build proper neural network, word build
Sentence from tf_Score latest works tend use non technical structure given task build proper neural network, word network
['following', 'list', 'researched', 'tasks', 'language', 'processing']
Sentence from tf_Score following list commonly researched tasks natural language processing, word following
Sentence from tf_Score following list commonly researched tasks natural language processing, word list
Sentence from tf_Score following list commonly researched tasks natural language processing, word researched
Sentence from tf_Score following list commonly researched tasks natural language processing, word task
Sentence from tf_Score following list commonly researched tasks natural language processing, word language
Sentence from tf_Score following list 

Sentence from tf_Score cognition refers mental action process acquiring knowledge understanding thought experience senses, word acquiring
Sentence from tf_Score cognition refers mental action process acquiring knowledge understanding thought experience senses, word knowledge
Sentence from tf_Score cognition refers mental action process acquiring knowledge understanding thought experience senses, word understanding
Sentence from tf_Score cognition refers mental action process acquiring knowledge understanding thought experience senses, word experience
Sentence from tf_Score cognition refers mental action process acquiring knowledge understanding thought experience senses, word sens
['science', 'study', 'mind', 'processes']
Sentence from tf_Score cognitive science interdisciplinary scientific study mind processes, word science
Sentence from tf_Score cognitive science interdisciplinary scientific study mind processes, word study
Sentence from tf_Score cognitive science interdisciplinary s

Sentence from tf_Score example george lakoff offers methodology build natural language processing nlp algorithms perspective cognitive science along findings cognitive linguistics two defining aspects ties cognitive linguistics part historical heritage nlp less frequently addressed since statistical turn, word cognitive
Sentence from tf_Score example george lakoff offers methodology build natural language processing nlp algorithms perspective cognitive science along findings cognitive linguistics two defining aspects ties cognitive linguistics part historical heritage nlp less frequently addressed since statistical turn, word linguistics
Sentence from tf_Score example george lakoff offers methodology build natural language processing nlp algorithms perspective cognitive science along findings cognitive linguistics two defining aspects ties cognitive linguistics part historical heritage nlp less frequently addressed since statistical turn, word part
Sentence from tf_Score example george

In [31]:
sentence_with_importance

[(6, 0.1835712902231117),
 (31, 0.17972807699436966),
 (42, 0.14121832790732206),
 (46, 0.13260926549769672),
 (36, 0.1314486896868196),
 (18, 0.12429995813005931),
 (29, 0.12393444709844863),
 (40, 0.12316039446217439),
 (7, 0.10533502877587728),
 (12, 0.10501322620967256),
 (34, 0.10299523104376769),
 (32, 0.10295374679075311),
 (15, 0.10016370125712329),
 (13, 0.09533021762514442),
 (45, 0.09361153677776775),
 (38, 0.09253478306045573),
 (11, 0.09099371135253255),
 (10, 0.08955583982687502),
 (24, 0.08833427421442845),
 (33, 0.08688081813365199),
 (14, 0.08523642987659971),
 (27, 0.08504034664131092),
 (44, 0.08163595036350423),
 (3, 0.08005910654169462),
 (1, 0.07969111414152937),
 (22, 0.0795239714904207),
 (9, 0.07940182946794154),
 (26, 0.07911723470002763),
 (37, 0.07750560159809973),
 (21, 0.07596230123211471),
 (39, 0.06880713630914215),
 (19, 0.06784015635244695),
 (4, 0.06634194395020268),
 (30, 0.06445523829596371),
 (41, 0.06103703287593189),
 (47, 0.06064453085948338),
 

In [38]:
input_user = int(input('Percentage of information to retain(in percent):'))
no_of_sentences = int((input_user * len(clean_sentences))/100)

cnt = 0
summary = [ ]
sentence_no = [ ]
for word_prob in sentence_with_importance:
    if cnt < no_of_sentences:
        sentence_no.append(word_prob[0])
        cnt = cnt+1
    else:
        break
sentence_no.sort()

cnt = 1
for sentence in sentences:
    if cnt in sentence_no:
        
        summary.append(sentence)
    cnt = cnt+1
summary = " ".join(summary)
print("\n")
print("Summary:")
print(summary)

Percentage of information to retain(in percent):20


Summary:
Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, a task that involves the automated interpretation and generation of natural language, but at the time not articulated as a problem separate from artificial intelligence. The premise of symbolic NLP is well-summarized by John Searle's Chinese room experiment: Given a collection of rules (e.g., a Chinese phrasebook, with questions and matching answers), the computer emulates natural language understanding (or other NLP tasks) by applying those rules to the data it is confronted with. These algorithms take as input a large set of "features" that are generated from the input data. Popular techniques include the use of word embeddings to capture semantic properties of words, and an increase in end-to-end learning of a higher-level task (e.g., question ans

In [41]:
print(keywords(summary,words=15,lemmatize=True))

learn
cognitive
answering
turing
language
machine translation
experience
intermediate
semantic
words
nlp
tasks
computing
titled
